<a href="https://colab.research.google.com/github/olaviinha/SloppyButchery/blob/main/util_AudioConcat.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#<font face="Trebuchet MS" size="6">Audio Concat <font color="#999" size="3">v 0.0.1<font color="#999" size="4">&nbsp;&nbsp;&nbsp;|&nbsp;&nbsp;&nbsp;</font><font size="4">Sloppy Butchery @</font> <a href="https://github.com/olaviinha/SloppyButchery" target="_blank"><font color="#999" size="4">Github</font></a><font color="#999" size="4">&nbsp;&nbsp;&nbsp;|&nbsp;&nbsp;&nbsp;</font><font size="3" color="#999"><a href="https://inha.se" target="_blank"><font color="#999">O. Inha</font></a></font></font>

Concatenates a directory of audio files in your Google Drive into one or more WAV files. Powered by [FFmpeg](https://github.com/FFmpeg/FFmpeg) and [SoX](https://sourceforge.net/p/sox/code/ci/master/tree/).

<hr size="1" color="#666"/>


In [ ]:
#@title #Setup
#@markdown This cell needs to be run only once. It will mount your Google Drive and setup prerequisities.

import os
from google.colab import output
force_setup = False

pip_packages = 'ffmpeg'

# inhagcutils
if not os.path.isfile('/content/inhagcutils.ipynb') and force_setup == False:
  %cd /content/
  !pip -q install import-ipynb {pip_packages}
  !curl -s -O https://raw.githubusercontent.com/olaviinha/inhagcutils/master/inhagcutils.ipynb
import import_ipynb
from inhagcutils import *

# Mount Drive
if not os.path.isdir('/content/drive') and force_setup == False:
  from google.colab import drive
  drive.mount('/content/drive')

# Drive symlink
if not os.path.isdir('/content/mydrive') and force_setup == False:
  os.symlink('/content/drive/My Drive', '/content/mydrive')
  drive_root_set = True
drive_root = '/content/mydrive/'

dir_tmp = '/content/tmp/'
create_dirs([dir_tmp])

import soundfile
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
nper = np.seterr(divide = 'ignore') 

!apt-get {apt_q} install sox
def chunker(seq, size):
  return (seq[i::size] for i in range(size))

output.clear()
op(c.ok, 'Setup finished.')


In [ ]:
#@title Concatenate

input_dir = "wavs" #@param {type:"string"}
output_file = "wavs/concat.wav" #@param {type:"string"}
#@markdown <small>Concatenate into this many evenly sized files (concat-1.wav, concat-2.wav, etc.)</small>
number_of_output_files = 1 #@param {type:"slider", min:1, max:10, step:1}

vol = 0.95 #@param {type:"slider", min:0, max:1, step:0.05}
concat_order = "Alphabetical" #@param ["Alphabetical", "Randomized"]
#@markdown <small>Convert all audio files before concatenation. This is required if your `input_dir` contains audio files in more than one format (e.g. 44.1 kHz and 48 kHz or 16 bit and 24 bit).</small>
convert = "44.1kHz_WAV" #@param ["None", "44.1kHz_WAV", "48kHz_WAV"]
#@markdown <small>Create audio player at the end to play the concatenated output_file. Use only if your output_file is going to be short, otherwise Colab will crash.</small>
preview = False #@param {type:"boolean"}

input_dir = fix_path(drive_root+input_dir)
output_file = drive_root+output_file

if os.path.isfile(output_file):
  !rm "{output_file}"
if not os.path.isdir(dir_tmp):
  dir_tmp = '/content/tmp/'
  dir_conv = dir_tmp+"conv/"
  create_dirs([dir_tmp, dir_conv])
if convert != 'None':
  !pip {pip_q} install ffmpeg

%cd {input_dir}
allfiles = list_audio(input_dir)
if convert != 'None':
  for i, file in enumerate(allfiles):
    input = str(file)
    output = dir_conv+'n'+str(i)+'.wav'
    if convert == '44.1kHz_WAV':
      conv_format = wav_44
    elif convert == '48kHz_WAV':
      conv_format = wav_48
    !ffmpeg {ffmpeg_q} -y -i "{input}" {conv_format} -af "pan=stereo|c0=c0|c1=c0" "{output}"
  allfiles = list_audio(dir_conv)
if concat_order is "Alphabetical":
  allfiles.sort()
else:
  random.shuffle(allfiles)
divider = math.ceil(len(allfiles)/number_of_output_files)
filelists = [allfiles[x:x+divider] for x in range(0, len(allfiles), divider)]
number_files = False
if len(filelists) > 1:
  number_files = True
print('Concat in progress...')
for i, filelist in enumerate(filelists):
  filelist_len = len(filelist)
  filelist = concat_list('-v '+str(vol), filelist)
  if number_files == True:
    output_file_f = path_dir(output_file)+basename(output_file)+'-'+str(i+1)+'.wav'
  !sox {filelist} "{output_file_f}"
  print(filelist_len, 'files concatenated to', output_file_f)
print('Done.')
if preview is True:
  Audio(output_file)